# DVS Set To Work

* Tiltmeter Checks
* Tiltmeter Calibration

                                                                Last updated 23/01/2026

In [ ]:
%matplotlib inline
import pylab as plt
import numpy as np

In [ ]:
import dvs
from analysis import katseops, katsepnt, katselib

In [ ]:
katselib.initsensorcache() # Init the local cache that speeds up katselib.getsensordata()

In [ ]:
ant = "e000"

In [ ]:
# From MKE-316-000000-ODC-TN-0083 the values in the config file for:
# 1. INCL_KS_ are simply the calibration sheet Ks values converted from ppm/degC to frac/degC
# 2. INCL_KZ_ are simply the calibration sheet Kz values converted from microrad/degC to degree/degC

uR2D = 1e-6*180/np.pi
uR2AS = uR2D*60*60

def json2katsepnt(cal):
    """ @param cal: a JSON dictionary exactly as what's in CustomerConfig.json
        @return: {cal_x=[], cal_y=[]} as used in katsepnt """
    # Extract as: [0, deg/V, frac/degC, arcsec, deg/degC, degC]
    x = [0] + [float(cal[_]) for _ in ["P050_SCALEINCL_X","P087_INCL_KS_X","P054_INCL_1_OFFSET_X","P085_INCL_KZ_X","P089_INCL_T_CAL_X"]]
    y = [0] + [float(cal[_]) for _ in ["P051_SCALEINCL_Y","P088_INCL_KS_Y","P055_INCL_1_OFFSET_Y","P086_INCL_KZ_Y","P090_INCL_T_CAL_Y"]]
    # Convert to: [0, microrad/V, microrad/V/degC, (negative)microrad, microrad/degC, degC
    x = [x[0], x[1]/uR2D, (x[1]/uR2D)*x[2], -x[3]/uR2AS, x[4]/uR2D, x[5]]
    y = [y[0], y[1]/uR2D, (y[1]/uR2D)*y[2], -y[3]/uR2AS, y[4]/uR2D, y[5]]
    return dict(cal_x=x, cal_y=y)

def katsepnt2json(cal):
    """ @param cal: {cal_x=[], cal_y=[]} as used in katsepnt
        @return: a JSON dictionary exactly as what's in CustomerConfig.json """
    x, y = cal['cal_x'], cal['cal_y']
    # Convert to: [0, deg/V, frac/degC, (negative)arcsec, deg/degC, degC]
    x = [x[0], x[1]*uR2D, x[2]/x[1], -x[3]*uR2AS, x[4]*uR2D, x[5]]
    y = [y[0], y[1]*uR2D, y[2]/y[1], -y[3]*uR2AS, y[4]*uR2D, y[5]]
    json = {}
    for k,v in zip(["P050_SCALEINCL_X","P087_INCL_KS_X","P054_INCL_1_OFFSET_X","P085_INCL_KZ_X","P089_INCL_T_CAL_X"], x[1:]):
        json[k] = "%g"%v
    for k,v in zip(["P051_SCALEINCL_Y","P088_INCL_KS_Y","P055_INCL_1_OFFSET_Y","P086_INCL_KZ_Y","P090_INCL_T_CAL_Y"], y[1:]):
        json[k] = "%g"%v
    return json


def sun_tilt(ants, Tstart, Tstop, debug=False):
    """ Compare tilt signals with other antennas, all in the same orientation, at sunrise, at >80degEl and Az=sunrise+/-10deg.
        
        Expect:
        1. tilt-x to "tip up": according to TBD tilt-x should become more positive.
        2. tilt-y is orthogonal to the temperature gradient, so should remain ~constant.
        3. slopes for both tilt-x and tilt-y should be the same for all antennas.
    """
    T = np.linspace(np.astype(Tstart,float), np.astype(Tstop,float), 1000)

    if debug: # Diagnostics
        katselib.plot_sensors(T, ["anc_wind_wind_speed"], figsize=(12,4))
        katselib.plot_sensors(T, ["%s_pos_actual_pointm_azim"%_ for _ in ants], figsize=(12,4))
        katselib.plot_sensors(T, ["%s_pos_actual_pointm_elev"%_ for _ in ants], figsize=(12,4));
    
    # Voltages
    katselib.plot_sensors(T, ["%s_dsm_tiltmeterXVoltage"%_ for _ in ants if _[0]!="m"]+["%s_ap_struclt_x"%_ for _ in ants if _[0]=="m"], fmt=lambda x: x-np.mean(x), figsize=(12,4))
    katselib.plot_sensors(T, ["%s_dsm_tiltmeterYVoltage"%_ for _ in ants if _[0]!="m"]+["%s_ap_struct_tilt_y"%_ for _ in ants if _[0]=="m"], fmt=lambda x: x-np.mean(x), figsize=(12,4));
    # Angles
    katselib.plot_sensors(T, ["%s_dsm_tiltmeterXCorr"%_ for _ in ants if _[0]!="m"]+["%s_ap_struct_tilt_x"%_ for _ in ants if _[0]=="m"], fmt=lambda x: x-np.mean(x), figsize=(12,4))
    katselib.plot_sensors(T, ["%s_dsm_tiltmeterYCorr"%_ for _ in ants if _[0]!="m"]+["%s_ap_struct_tilt_y"%_ for _ in ants if _[0]=="m"], fmt=lambda x: x-np.mean(x), figsize=(12,4))

## Tiltmeter Checks

In [ ]:
katselib.plot_sensors([np.datetime64("2025-11-20 00:00:00").astype(float), np.datetime64("2025-12-20 20:00:00").astype(float), 1], # 1 sec sampling
                         [ant+"_dsm_tiltmeterTemp"], figsize=(12,5));

In [ ]:
katselib.plot_sensors([np.datetime64("2025-12-20 00:00:00").astype(float), np.datetime64("2026-01-20 20:00:00").astype(float), 1], # 1 sec sampling
                         [ant+"_dsm_tiltmeterTemp"], figsize=(12,5));

### Sunrise Test
This test was useful for MeerKAT dishes: balanced in elevation, so little change in tilt over elevation.


For MKE dishes tilt-x & tilt-y can also be distinguished by simply scanning in elevation: tilt-y should remain constant while tilt-x should decrease with increasing elevation.

In [ ]:
sun_tilt([ant,REF_TBD], np.datetime64("2026-01-TBD 04:30", 's'), np.datetime64("2026-01-TBD 06:00", 's'), debug=False)

In [ ]:
# EXPECT: Orientation & signs of X & Y exactly match m000 - m063 and/or MKE121 (which was shown to match the MeerKAT convention).

### Calibration Measurements - AS-IS
Regular measurements used to monitor status and to update calibration if necessary. This uses data as calibrated by the ACU.

In [ ]:
# The output file naming patterns
!ls -la ./l1_data/*.zip

In [ ]:
# Download the data from the sensor database
file_tag = "pedestal-tilt-%s.pkl.zip"
# Specify calibration coefficients to use tiltxraw & tiltyraw rather than the ACU's tiltx & tilty
if (ant in katsepnt.TILT_ACTIVE_CAL.keys()): del katsepnt.TILT_ACTIVE_CAL[ant]

# Time interval during which ACU coefficients were constant
## OHB's coefficients
Tstart = np.datetime64('2025-11-20 20:00:00', 's').astype(int)
Tstop = np.datetime64('2026-01-19 20:00:00', 's').astype(int)
## Tempco's & offsets update
# Tstart = np.datetime64('2026-01-22 14:00:00', 's').astype(int)
# Tstop = np.datetime64('2026-11-09 00:00:00', 's').astype(int)

tilt_ds = katsepnt.get_tilt_data((Tstart, Tstop), ants=[ant])
katsepnt.summary_plot_rawtilt(tilt_ds) # One figure per receptor, may be useful for debugging

# Combine with earlier data, if it exists, to load incrementally
#tilt_ds = katseops.merge_dictdatasets(katseops.load_tilt_ds(file_tag % ant, cache_root="./l1_data"), tilt_ds)
# Save the dataset
tiltDBfile = katseops.save_tilt_ds(tilt_ds, file_tag % ant, cache_root="./l1_data")

### Assess Status
Use this section to continuously assess the status of the sensors based on specific monitoring measurements.

Use visual assessment to identify if anything seems strange, or if calibration needs to be updated.

#### Tiltmeter Fits

In [ ]:
file_tag = "pedestal-tilt-%s.pkl.zip"
tilt_ds = katseops.load_tilt_ds(file_tag%ant, cache_root="./l1_data")

In [ ]:
# Day & night time
katsepnt.fit_tiltmeasurements(tilt_ds[ant], ant, mask_sun=False, mask_wind=4.0, # m/s. For MeerKAT wind>3.0 results in significant systematic changes!
                          harmonics=True, # False for "model fit"
                          w_key="wind", w_sense="inverse", Az=np.linspace(-135,225,360),
                          summary_plot=True, verbose=True);

In [ ]:
# Only measurements between sunset till sunrise
fit = katsepnt.fit_tiltmeasurements(tilt_ds[ant], ant, mask_sun=True, mask_wind=4.0, # m/s. For MeerKAT wind>3.0 results in significant systematic changes!
                          harmonics=True, # False for "model fit"
                          w_key="wind", w_sense="inverse", Az=np.linspace(-135,225,360),
                          summary_plot=True, verbose=True)

In [ ]:
# Only measurements between sunset till sunrise, and exclude non-typical elevation angles
tilt_el = np.array([np.mean(_) for _ in tilt_ds[ant]['el']])
EL0 = np.abs(tilt_el - np.mean(tilt_el)) < 10
fit = katsepnt.fit_tiltmeasurements(tilt_ds[ant], ant, mask=EL0, mask_sun=True, mask_wind=4.0, # m/s. For MeerKAT wind>3.0 results in significant systematic changes!
                          harmonics=True, # False for "model fit"
                          w_key="wind", w_sense="inverse", Az=np.linspace(-135,225,360),
                          summary_plot=True, verbose=True)

In [ ]:
# -> Night time AN0, AW0 seem reasonably stable. Also Y offset stable.
# -> X offset jumps 15arcsec on one night and drifts a lot - temperature coefficients or something wrong?

#### Temperature Coefficients
Use `fit_tiltmetertempcos` to visualise evidence for change to temperature coefficients.

1. fit_tiltmetertempcos() yields the following terms:
```
      p_TiltOffset = -( actual_ZeroShift - acu_ZeroShift + acu_TiltOffset ) [arcsec]
      p_TiltOffsetTempCo = -( actual_ZeroShiftTempCo - acu_ZeroShiftTempCo ) [arcsec/degC]
```
3. The updated ACU parameter values should be:
```
      new_ZeroShiftTempCo <== acu_ZeroShiftTempCo - p_TiltOffsetTempCo
      new_TiltOffset <== acu_TiltOffset + p_TiltOffset
```
but use `fit_tiltmeasurements(..., recal=...)` to formally derive new values.

In [ ]:
NIGHT = np.array([katselib.is_nighttime(t.min()) for t in tilt_ds[ant]["TS"]])
tilt_el = np.array([np.mean(_) for _ in tilt_ds[ant]['el']])
EL0 = np.abs(tilt_el - np.mean(tilt_el)) < 10

In [ ]:
p_ox, p_oy, sigma_ox, sigma_oy, p_AN, p_AW, sigma_AN, sigma_AW = katsepnt.fit_tiltmetertempcos(
                 tilt_ds[ant], ant, mask_wind=4, fitmasks=[None,NIGHT&EL0], detrend=0, fitANAW=True, verbose=False)

In [ ]:
# For info, plot CW & CCW data separately
CW = np.array([(az[3]-az[0]>0) for az in tilt_ds[ant]["az"]]) # For daytime data, CW & CCW rotation give different 'ox' for MeerKAT!
_ = katsepnt.fit_tiltmetertempcos(tilt_ds[ant], ant, mask_wind=4, # mask_wind=10 for info, otherwise use 4.0!
                              fitmasks=[CW,~CW], detrend=0, fitANAW=True, verbose=False)

In [ ]:
# -> The above provides clear evidence that X offset tempco must be updated; Y tempco is ~1 sigma

# Manual update (check) from the solution found above.
# The data above was scaled by the ACU using Customer_Config.json (updated 24/11/2025) 
cal = {
      "P050_SCALEINCL_X": "3.0309467e-3",
      "P051_SCALEINCL_Y": "3.0624594e-3",
      "P054_INCL_1_OFFSET_X": "0.0",
      "P055_INCL_1_OFFSET_Y": "0.0",
      "P085_INCL_KZ_X": "-3.2618487e-4",
      "P086_INCL_KZ_Y": "5.3571554e-4",
      "P087_INCL_KS_X": "2.7832687e-3",
      "P088_INCL_KS_Y": "-3.7388669e-4",
      "P089_INCL_T_CAL_X": "27.90",
      "P090_INCL_T_CAL_Y": "27.60",
}
acu_ZeroShiftTempCo = np.r_[float(cal["P085_INCL_KZ_X"]), float(cal["P086_INCL_KZ_Y"])] # degree/degC; 

p_TiltOffsetTempCo = np.r_[p_ox[1], p_oy[1]] # arcsec/degC
new_ZeroShiftTempCo = acu_ZeroShiftTempCo - p_TiltOffsetTempCo*(1/(60*60)) # [degree/degC]
print("Expect KZ <==", new_ZeroShiftTempCo)

In [ ]:
# Best updated set of coefficients should be found from formal fits:
# NB: `fit_tiltmeasurements(tilt_ds, recal)` assumes tilt_ds was generated using what's in katsepnt.TILT_ACTIVE_CAL[ant]!
katsepnt.TILT_ACTIVE_CAL[ant] = json2katsepnt(cal)

NIGHT = [katselib.is_nighttime(np.min(_),horizon=-15) for _ in tilt_ds[ant]['TS']]
katsepnt.fit_tiltmeasurements(tilt_ds[ant], ant, mask=NIGHT&EL0, mask_wind=4.0, # m/s. For MeerKAT wind>3.0 results in significant systematic changes!
                          recal=2, # Fit only scale factor tempco (KS)
                          w_key="wind", w_sense="inverse", Az=np.linspace(-135,225,360), harmonics=True,
                          summary_plot=True, verbose=False);
katsepnt.fit_tiltmeasurements(tilt_ds[ant], ant, mask=NIGHT&EL0, mask_wind=4.0, # m/s. For MeerKAT wind>3.0 results in significant systematic changes!
                          recal=3, # Fit only zero offset tempco (KZ)
                          w_key="wind", w_sense="inverse", Az=np.linspace(-135,225,360),
                          summary_plot=True, verbose=False);
katsepnt.fit_tiltmeasurements(tilt_ds[ant], ant, mask=NIGHT&EL0, mask_wind=4.0, # m/s. For MeerKAT wind>3.0 results in significant systematic changes!
                          recal=1, # Fit all tempcos (KS & KZ)
                          w_key="wind", w_sense="inverse", Az=np.linspace(-135,225,360),
                          summary_plot=True, verbose=False);

In [ ]:
# -> recal=1 (KS & KZ) gives lowest residuals and by eye seems to stabilize everything to < 5arcsec, so use that?
# -> HOWEVER, temperature range is too limited - prefer to change minimum necessary parameters i.e. recal=3 (KZ)
recal = dict(
  # Manually copied from above
  cal_x=[0, 52.899999367456815, 0.14723491246946235, 0.0, np.float64(2.7837081554446876), 27.9],
  cal_y=[0, 53.44999973865004, -0.019984243482784728, 0.0, np.float64(5.479608686836212), 27.6]
)
# Manually added from above, as NEGATIVE of what's printed!
recal["cal_x"][-3], recal["cal_y"][-3] = (-9.39656/uR2AS*-1, -36.4055/uR2AS*-1)


print("Expect KZ <==", new_ZeroShiftTempCo) # "manual check" earlier
katsepnt2json(recal)

### Double-check Before Update
Use this section to anticipate what the effect would be of a change to the calibration data

In [ ]:
# Solution found above
katsepnt.TILT_ACTIVE_CAL[ant] = recal

In [ ]:
# Download the data from the sensor database, retro-fit the updated coefficients
file_tag = "pedestal-tilt-modKz-%s.pkl.zip"

Tstart = np.datetime64('2025-11-20 20:00:00', 's').astype(int)
Tstop = np.datetime64('2026-01-19 20:00:00', 's').astype(int)

tilt_ds = katsepnt.get_tilt_data((Tstart, Tstop), ants=[ant])
katsepnt.summary_plot_rawtilt(tilt_ds) # One figure per receptor, may be useful for debugging

# Combine with earlier data, if it exists
#tilt_ds = katseops.merge_dictdatasets(katseops.load_tilt_ds(file_tag % ant, cache_root="./l1_data"), tilt_ds)
# Save the dataset
tiltDBfile = katseops.save_tilt_ds(tilt_ds, file_tag % ant, cache_root="./l1_data")

In [ ]:
# With cal updated - day & night time
file_tag = "pedestal-tilt-modKz-%s.pkl.zip"
tilt_ds = katseops.load_tilt_ds(file_tag%ant, cache_root="./l1_data")

fit = katsepnt.fit_tiltmeasurements(tilt_ds[ant], ant, mask_sun=False, mask_wind=4.0, # m/s. For MeerKAT wind>3.0 results in significant systematic changes!
                          harmonics=True, # False for "model fit"
                          w_key="wind", w_sense="inverse", Az=np.linspace(-135,225,360),
                          summary_plot=True, verbose=True)

In [ ]:
# Only measurements between sunset till sunrise, and exclude non-typical elevation angles
NIGHT = np.array([katselib.is_nighttime(t.min()) for t in tilt_ds[ant]["TS"]])
tilt_el = np.array([np.mean(_) for _ in tilt_ds[ant]['el']])
EL0 = np.abs(tilt_el - np.mean(tilt_el)) < 10

p_ox, p_oy, sigma_ox, sigma_oy, p_AN, p_AW, sigma_AN, sigma_AW = katsepnt.fit_tiltmetertempcos(
                 tilt_ds[ant], ant, fitANAW=True, mask_wind=4, fitmasks=[None,NIGHT&EL0], detrend=0, verbose=False)

If the above result is positive, update the ACU & continue collecting more monitoring data.

### Check Model Prediction Accuracy

In [ ]:
# Data to use to construct the model
file_tag = "pedestal-tilt-modKz-%s.pkl.zip"
tilt_ds = katseops.load_tilt_ds(file_tag%ant, cache_root="./l1_data")
katsepnt.TILT_ACTIVE_CAL[ant] = recal # Needed for katsepnt.get_tiltxy() further below, to re-compute from raw tilt

In [ ]:
def modeltilt(az, el, ox,oy,AN,AW, A2=0,P2=0, doxdel=0):
    """ This is ALMOST the model used by katsepnt.fit_tiltmeasurements except that:
        1. temperature compensation is omitted, for clarity
        2. the signs of the coefficients are swapped as necessary to represent **measurements** not **corrections**
        3. new term 'doxdel' (default 0) is added as explained below in this notebook.
        
        @param az, el: array of angles [deg]
        @param ox,oy,AN0,AW0: first order model of the tiltmeter inside the pedestal [arcsec]
        @param A2,P2: second order coefficients for the azimuth axis [arcsec, deg] (default 0)
        @param doxdel: imbalance change in ox relative to the value at 50degEl [arcsec/degEl] (default 0)
        @return: (tiltx, tilty) model tilt [arcsec, arcsec] """
    # The new bit
    el = np.squeeze(np.reshape(el,(-1,1)))
    ox = ox + doxdel*(el-50)
    # The simplified old bit
    az_n = np.squeeze(np.reshape(az,(-1,1)))/360.
    x = abs(A2)*np.cos(2*np.pi*(2*az_n)-P2*np.pi/180) - ox - AN*np.cos(2*np.pi*az_n) + AW*np.sin(2*np.pi*az_n)
    y = abs(A2)*np.sin(2*np.pi*(2*az_n)-P2*np.pi/180) - oy - AN*np.sin(2*np.pi*az_n) - AW*np.cos(2*np.pi*az_n)
    return x, y

In [ ]:
## Determine the unbalance impact on tiltx over elevation
# Tilt measurements made NOT at the typical 50degEl
tilt_el = np.array([np.mean(_) for _ in tilt_ds[ant]['el']])
EL_d = np.abs(tilt_el - np.mean(tilt_el)) > 10
# Add a few which are close to mean, try to get some with low wind
EL_d[np.flatnonzero(~EL_d)[:2]] = True
EL_d[np.flatnonzero(~EL_d)[-3:]] = True

ox, oy, an0, aw0, a2, p2, *_ = katsepnt.fit_tiltmeasurements(tilt_ds[ant], ant, mask=EL_d, w_key='wind', verbose=True); plt.close('all')
tilt_el = tilt_el[EL_d]
print("\nElevation angles:", tilt_el)

plt.figure(figsize=(8,4))
plt.plot(tilt_el, ox[1:], '|', label="ox") # [0] is the ensamble average
plt.plot(tilt_el, oy[1:], '_', label="oy")
plt.legend()

print("\ndox/del:")
print((np.diff(ox[1:])/np.diff(tilt_el))[np.abs(np.diff(tilt_el))>1], "arcsec/degEl")

In [ ]:
## Get the remaining model coefficients
NIGHT = np.array([katselib.is_nighttime(t.min()) for t in tilt_ds[ant]["TS"]])
tilt_el = np.array([np.mean(_) for _ in tilt_ds[ant]['el']])
EL0 = np.abs(tilt_el - np.mean(tilt_el)) < 10

ox, oy, an0, aw0, a2, p2, *_ = katsepnt.fit_tiltmeasurements(tilt_ds[ant], ant, mask=NIGHT&EL0, mask_wind=4, w_key='wind', verbose=True); plt.close('all')

In [ ]:
# Pick a time interval of interest
TS = np.arange(np.datetime64("2026-01-14 01:15", 's').astype(float), np.datetime64("2026-01-14 01:45", 's').astype(float), 1) # Tilt cal


In [ ]:
# Get the actual recorded angles
az, el = [katselib.getsensorvalues("%s_%s"%(ant,katsepnt._TILT_CORR_KEYS_(ant)[_]['key']), TS, interpolate='linear')[1] for _ in ['az','el']] # deg, deg
# NB: If there's a post-facto update for the coefficients then this re-computes the tilt angles from "raw tilt"
tiltx, tilty = katsepnt.get_tiltxy(TS, ant, **katsepnt.TILT_ACTIVE_CAL[ant])[1:] # arcsec, arcsec

# The expected tilt at the az,el, from the tilt model. Use [0] for ensamble mean or [-1] for latest
mx, my = modeltilt(az, el, ox[0], oy[0], an0[0], aw0[0], a2[0], p2[0]*180/np.pi, doxdel=-0.6)

In [ ]:
# Compare the recorded tilt to the model tilt
axs = plt.subplots(2,2, sharey='row', figsize=(12,6))[1]

axs[0][0].plot(TS, tiltx, 'x'); axs[0][0].plot(TS, mx, '-')
axs[0][0].set_ylabel("tilt X [arcsec]")
axs[0][1].plot(TS, tilty, 'x'); axs[0][1].plot(TS, my, '-')
axs[0][1].set_ylabel("tilt Y [arcsec]")

# These residuals should be zero mean, except for sun and wind influence
axs[1][0].plot(TS, tiltx-mx, '|', label="X")
axs[1][0].plot(TS, tilty-my, '_', label="Y")
axs[1][0].set_ylabel("tilt-model [arcsec]"); axs[1][0].legend(); axs[1][0].set_ylim(-40,40)
axs[1][1].hist(tiltx-mx, range=(-30,30), orientation='horizontal', density=True, alpha=0.6, label="X")
axs[1][1].hist(tilty-my, range=(-30,30), orientation='horizontal', density=True, alpha=0.6, label="Y"); axs[1][1].legend();